# Sistema de Informação de Agravos de Notificação (SINAN)

Esse notebook tem como objetivo realizar o ELT dos dados provenientes do SINAN que serão utilizados no projeto Dengue.

### Libs

In [1]:
%run ../../config/bootstrap.py

In [2]:
import pandas as pd
from datetime import datetime 
from utils import get_project_root, save_parquet_in_chunks  
import basedosdados as bd

In [3]:
project_root = get_project_root() 
#billing_id = 'ufrgs-ppgc-dengue'

In [4]:
#%load_ext sql
%reload_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.displaylimit = 100
%sql bigquery://ufrgs-ppgc-dengue

Connecting to 'bigquery://ufrgs-ppgc-dengue'

# ⌛ Staging
 

In [5]:
%%sql
SELECT * FROM `basedosdados.br_ms_sinan.dicionario` limit 10

Running query in 'bigquery://ufrgs-ppgc-dengue'

10 rows affected.

,id_tabela,nome_coluna,chave,cobertura_temporal,valor
0,microdados_dengue,escolaridade_paciente,1,(1),1ª a 4ª série incompleta do EF
1,microdados_dengue,escolaridade_paciente,2,(1),4ª série completa do EF ( antigo 1° grau)
2,microdados_dengue,escolaridade_paciente,3,(1),5ª à 8ª série incompleta do EF (antigo ginásio ou 1° grau)
3,microdados_dengue,raca_cor_paciente,3,(1),Amarela
4,microdados_dengue,idade_paciente,4,(1),Ano
5,microdados_dengue,raca_cor_paciente,1,(1),Branca
6,microdados_dengue,criterio_confirmacao,2,(1),Clínico Epidemiológico
7,microdados_dengue,evolucao_caso,1,(1),Cura
8,microdados_dengue,idade_paciente,2,(1),Dia
9,microdados_dengue,escolaridade_paciente,8,(1),Educação superior completa


In [6]:
#df_sinan_dic.to_csv(project_root / "data/staging/sinan/dicionario.txt", sep = ";", index = False)

# notifications

In [7]:
%%sql  
SELECT ano, count(*) as qt FROM `basedosdados.br_ms_sinan.microdados_dengue` group by 1 order by 1 

Running query in 'bigquery://ufrgs-ppgc-dengue'

26 rows affected.

,ano,qt
0,2000,172855
1,2001,488590
2,2002,897093
3,2003,416609
4,2004,136867
5,2005,261501
6,2006,411022
7,2007,717097
8,2008,919324
9,2009,600658


# bi_query

### 2021

In [8]:
%%sql df <<
SELECT 
	tipo_notificacao
        ,id_agravo
        ,data_notificacao
        --,semana_notificacao
        --,sigla_uf_notificacao
        --,sigla_uf_notificacao_nome
        ,id_regional_saude_notificacao
        ,id_municipio_notificacao
        --,id_municipio_notificacao_nome
        ,id_estabelecimento as id_cnes
        ,data_primeiros_sintomas
        ,semana_sintomas
        ,pais_residencia
        --,sigla_uf_residencia
        --,sigla_uf_residencia_nome
        --,id_regional_saude_residencia
        --,id_municipio_residencia
        --,id_municipio_residencia_nome
        ,ano_nascimento_paciente
        ,data_nascimento_paciente
        ,idade_paciente
        ,sexo_paciente
        ,raca_cor_paciente
        ,escolaridade_paciente
        ,ocupacao_paciente 
        ,gestante_paciente
        ,possui_doenca_autoimune
        ,possui_diabetes
        ,possui_doencas_hematologicas
        ,possui_hepatopatias
        ,possui_doenca_renal
        ,possui_hipertensao
        ,possui_doenca_acido_peptica
        ,paciente_vacinado
        --,data_vacina
        ,data_investigacao
        ,apresenta_febre
        ,data_febre
        ,duracao_febre
        ,apresenta_cefaleia
        ,apresenta_exantema
        ,apresenta_dor_costas
        ,apresenta_prostacao
        ,apresenta_mialgia
        ,apresenta_vomito
        ,apresenta_nausea
        ,apresenta_diarreia
        ,apresenta_conjutivite
        ,apresenta_dor_retroorbital
        ,apresenta_artralgia
        ,apresenta_artrite
        ,apresenta_leucopenia
        ,apresenta_epistaxe
        ,apresenta_petequias
        ,apresenta_gengivorragia
        ,apresenta_metrorragia
        ,apresenta_hematuria
        ,apresenta_sangramento
        ,apresenta_complicacao
        ,apresenta_ascite
        ,apresenta_pleurite
        ,apresenta_pericardite
        ,apresenta_dor_abdominal
        ,apresenta_hepatomegalia
        ,apresenta_miocardite
        ,apresenta_hipotensao
        ,apresenta_choque
        ,apresenta_insuficiencia_orgao
        ,apresenta_sintoma_outro
        ,apresenta_qual_sintoma
        ,prova_laco
        ,internacao
        ,data_internacao
        --,sigla_uf_internacao
        --,sigla_uf_internacao_nome
        ,id_municipio_internacao
        --,id_municipio_internacao_nome
        ,alarme_hipotensao
        ,alarme_plaqueta
        ,alarme_vomito
        ,alarme_sangramento
        ,alarme_hematocrito
        ,alarme_dor_abdominal
        ,alarme_letargia
        ,alarme_hepatomegalia
        ,alarme_liquidos
        ,data_alarme
        ,grave_pulso
        ,grave_convulsao
        ,grave_enchimento_capilar
        ,grave_insuficiencia_respiratoria
        ,grave_taquicardia
        ,grave_extremidade_fria
        ,grave_hipotensao
        ,grave_hematemese
        ,grave_melena
        ,grave_metrorragia
        ,grave_sangramento
        ,grave_ast_alt
        ,grave_miocardite
        ,grave_consciencia
        ,grave_orgaos
        ,data_hematocrito
        ,hematocrito_maior
        ,data_plaquetas
        ,plaqueta_maior
        ,data_hematocrito_2
        ,hematocrito_menor
        ,data_plaquetas_2
        ,plaqueta_menor
        ,data_sorologia1_chikungunya
        ,data_resultado_sorologia1_chikungunya
        ,resultado_sorologia1_chikungunya
        ,sorologia1_igm
        ,sorologia1_igg
        ,sorologia1_tit1
        ,resultado_sorologia2_chikungunya
        ,sorologia2_igm
        ,sorologia2_igg
        ,sorologia2_tit1
        ,resultado_prnt
        ,data_ns1
        ,resultado_ns1
        ,data_viral
        ,resultado_viral
        ,data_pcr
        ,resultado_pcr
        ,amostra_pcr
        ,amostra_outra
        ,tecnica
        ,resultado_amostra_outra
        ,data_sorologia_dengue
        ,resultado_sorologia_dengue
        ,sorotipo
        ,histopatologia
        ,imunohistoquimica
        ,manifestacao_hemorragica
        ,classificacao_final
        ,criterio_confirmacao
        ,caso_fhd
        ,caso_autoctone
        ,pais_infeccao
        --,sigla_uf_infeccao
        --,sigla_uf_infeccao_nome
        ,id_municipio_infeccao
        --,id_municipio_infeccao_nome
        ,doenca_trabalho
        ,apresentacao_clinica
        ,evolucao_caso
        --,data_obito
        --,data_encerramento
        ,tipo_sistema
        --,data_digitacao
,  (CASE WHEN LOWER(classificacao_final) LIKE '%dengue%' THEN 1 ELSE 0 END) AS target
,  (CASE WHEN LOWER(evolucao_caso) LIKE '%óbito%' THEN 1 ELSE 0 END) AS obito
,  (CASE WHEN data_encerramento is not null THEN 1  ELSE 0  END) AS flag_caso_encerrado

---Dias
, (CASE 
        WHEN data_encerramento is not null and data_encerramento>=coalesce(data_primeiros_sintomas,data_notificacao)
        THEN  DATE_DIFF(data_encerramento, coalesce(data_primeiros_sintomas,data_notificacao), DAY)
        ELSE null  END) as qt_dias_encerramento
, (CASE 
        WHEN data_obito is not null
        THEN  DATE_DIFF(data_obito, coalesce(data_primeiros_sintomas,data_notificacao), DAY)
        ELSE null  END) as qt_dias_obito
, (CASE 
        WHEN data_primeiros_sintomas is not null
        THEN  DATE_DIFF(data_primeiros_sintomas, data_notificacao, DAY)
        ELSE null  END) as qt_dias_notificacao_primeiros_sintomas
,count(*) as qt

from `basedosdados.br_ms_sinan.microdados_dengue`
where 1=1
and ano=2021
group by 
                tipo_notificacao
                ,id_agravo
                ,data_notificacao
                --,semana_notificacao
                --,sigla_uf_notificacao
                --,sigla_uf_notificacao_nome
                ,id_regional_saude_notificacao
                ,id_municipio_notificacao
                --,id_municipio_notificacao_nome
                ,id_estabelecimento
                ,data_primeiros_sintomas
                ,semana_sintomas
                ,pais_residencia
                --,sigla_uf_residencia
                --,sigla_uf_residencia_nome
                --,id_regional_saude_residencia
                --,id_municipio_residencia
                --,id_municipio_residencia_nome
                ,ano_nascimento_paciente
                ,data_nascimento_paciente
                ,idade_paciente
                ,sexo_paciente
                ,raca_cor_paciente
                ,escolaridade_paciente
                ,ocupacao_paciente 
                ,gestante_paciente
                ,possui_doenca_autoimune
                ,possui_diabetes
                ,possui_doencas_hematologicas
                ,possui_hepatopatias
                ,possui_doenca_renal
                ,possui_hipertensao
                ,possui_doenca_acido_peptica
                ,paciente_vacinado
                --,data_vacina
                ,data_investigacao
                ,apresenta_febre
                ,data_febre
                ,duracao_febre
                ,apresenta_cefaleia
                ,apresenta_exantema
                ,apresenta_dor_costas
                ,apresenta_prostacao
                ,apresenta_mialgia
                ,apresenta_vomito
                ,apresenta_nausea
                ,apresenta_diarreia
                ,apresenta_conjutivite
                ,apresenta_dor_retroorbital
                ,apresenta_artralgia
                ,apresenta_artrite
                ,apresenta_leucopenia
                ,apresenta_epistaxe
                ,apresenta_petequias
                ,apresenta_gengivorragia
                ,apresenta_metrorragia
                ,apresenta_hematuria
                ,apresenta_sangramento
                ,apresenta_complicacao
                ,apresenta_ascite
                ,apresenta_pleurite
                ,apresenta_pericardite
                ,apresenta_dor_abdominal
                ,apresenta_hepatomegalia
                ,apresenta_miocardite
                ,apresenta_hipotensao
                ,apresenta_choque
                ,apresenta_insuficiencia_orgao
                ,apresenta_sintoma_outro
                ,apresenta_qual_sintoma
                ,prova_laco
                ,internacao
                ,data_internacao
                --,sigla_uf_internacao
                --,sigla_uf_internacao_nome
                ,id_municipio_internacao
                --,id_municipio_internacao_nome
                ,alarme_hipotensao
                ,alarme_plaqueta
                ,alarme_vomito
                ,alarme_sangramento
                ,alarme_hematocrito
                ,alarme_dor_abdominal
                ,alarme_letargia
                ,alarme_hepatomegalia
                ,alarme_liquidos
                ,data_alarme
                ,grave_pulso
                ,grave_convulsao
                ,grave_enchimento_capilar
                ,grave_insuficiencia_respiratoria
                ,grave_taquicardia
                ,grave_extremidade_fria
                ,grave_hipotensao
                ,grave_hematemese
                ,grave_melena
                ,grave_metrorragia
                ,grave_sangramento
                ,grave_ast_alt
                ,grave_miocardite
                ,grave_consciencia
                ,grave_orgaos
                ,data_hematocrito
                ,hematocrito_maior
                ,data_plaquetas
                ,plaqueta_maior
                ,data_hematocrito_2
                ,hematocrito_menor
                ,data_plaquetas_2
                ,plaqueta_menor
                ,data_sorologia1_chikungunya
                ,data_resultado_sorologia1_chikungunya
                ,resultado_sorologia1_chikungunya
                ,sorologia1_igm
                ,sorologia1_igg
                ,sorologia1_tit1
                ,resultado_sorologia2_chikungunya
                ,sorologia2_igm
                ,sorologia2_igg
                ,sorologia2_tit1
                ,resultado_prnt
                ,data_ns1
                ,resultado_ns1
                ,data_viral
                ,resultado_viral
                ,data_pcr
                ,resultado_pcr
                ,amostra_pcr
                ,amostra_outra
                ,tecnica
                ,resultado_amostra_outra
                ,data_sorologia_dengue
                ,resultado_sorologia_dengue
                ,sorotipo
                ,histopatologia
                ,imunohistoquimica
                ,manifestacao_hemorragica
                ,classificacao_final
                ,criterio_confirmacao
                ,caso_fhd
                ,caso_autoctone
                ,pais_infeccao
                --,sigla_uf_infeccao
                --,sigla_uf_infeccao_nome
                ,id_municipio_infeccao
                --,id_municipio_infeccao_nome
                ,doenca_trabalho
                ,apresentacao_clinica
                ,evolucao_caso
                ,data_obito
                ,data_encerramento
                ,tipo_sistema
                --,data_digitacao
        ,qt_dias_encerramento
        ,qt_dias_obito
        ,qt_dias_notificacao_primeiros_sintomas


Running query in 'bigquery://ufrgs-ppgc-dengue'

1009779 rows affected.

In [9]:
save_parquet_in_chunks(df,"data/4_gold/sinan/fat_sinan/2021",50,compression='brotli')

Salvando arquivos Parquet:   0%|          | 0/1009779 [00:00<?, ?linhas/s]

✔️ 2021_part_1.parquet salvo com 15.76 MB (1009779 linhas)
✅ Todos os arquivos salvos com sucesso.


## 2022

In [10]:
%%sql df <<
SELECT 
	tipo_notificacao
        ,id_agravo
        ,data_notificacao
        --,semana_notificacao
        --,sigla_uf_notificacao
        --,sigla_uf_notificacao_nome
        ,id_regional_saude_notificacao
        ,id_municipio_notificacao
        --,id_municipio_notificacao_nome
        ,id_estabelecimento as id_cnes
        ,data_primeiros_sintomas
        ,semana_sintomas
        ,pais_residencia
        --,sigla_uf_residencia
        --,sigla_uf_residencia_nome
        --,id_regional_saude_residencia
        --,id_municipio_residencia
        --,id_municipio_residencia_nome
        ,ano_nascimento_paciente
        ,data_nascimento_paciente
        ,idade_paciente
        ,sexo_paciente
        ,raca_cor_paciente
        ,escolaridade_paciente
        ,ocupacao_paciente 
        ,gestante_paciente
        ,possui_doenca_autoimune
        ,possui_diabetes
        ,possui_doencas_hematologicas
        ,possui_hepatopatias
        ,possui_doenca_renal
        ,possui_hipertensao
        ,possui_doenca_acido_peptica
        ,paciente_vacinado
        --,data_vacina
        ,data_investigacao
        ,apresenta_febre
        ,data_febre
        ,duracao_febre
        ,apresenta_cefaleia
        ,apresenta_exantema
        ,apresenta_dor_costas
        ,apresenta_prostacao
        ,apresenta_mialgia
        ,apresenta_vomito
        ,apresenta_nausea
        ,apresenta_diarreia
        ,apresenta_conjutivite
        ,apresenta_dor_retroorbital
        ,apresenta_artralgia
        ,apresenta_artrite
        ,apresenta_leucopenia
        ,apresenta_epistaxe
        ,apresenta_petequias
        ,apresenta_gengivorragia
        ,apresenta_metrorragia
        ,apresenta_hematuria
        ,apresenta_sangramento
        ,apresenta_complicacao
        ,apresenta_ascite
        ,apresenta_pleurite
        ,apresenta_pericardite
        ,apresenta_dor_abdominal
        ,apresenta_hepatomegalia
        ,apresenta_miocardite
        ,apresenta_hipotensao
        ,apresenta_choque
        ,apresenta_insuficiencia_orgao
        ,apresenta_sintoma_outro
        ,apresenta_qual_sintoma
        ,prova_laco
        ,internacao
        ,data_internacao
        --,sigla_uf_internacao
        --,sigla_uf_internacao_nome
        ,id_municipio_internacao
        --,id_municipio_internacao_nome
        ,alarme_hipotensao
        ,alarme_plaqueta
        ,alarme_vomito
        ,alarme_sangramento
        ,alarme_hematocrito
        ,alarme_dor_abdominal
        ,alarme_letargia
        ,alarme_hepatomegalia
        ,alarme_liquidos
        ,data_alarme
        ,grave_pulso
        ,grave_convulsao
        ,grave_enchimento_capilar
        ,grave_insuficiencia_respiratoria
        ,grave_taquicardia
        ,grave_extremidade_fria
        ,grave_hipotensao
        ,grave_hematemese
        ,grave_melena
        ,grave_metrorragia
        ,grave_sangramento
        ,grave_ast_alt
        ,grave_miocardite
        ,grave_consciencia
        ,grave_orgaos
        ,data_hematocrito
        ,hematocrito_maior
        ,data_plaquetas
        ,plaqueta_maior
        ,data_hematocrito_2
        ,hematocrito_menor
        ,data_plaquetas_2
        ,plaqueta_menor
        ,data_sorologia1_chikungunya
        ,data_resultado_sorologia1_chikungunya
        ,resultado_sorologia1_chikungunya
        ,sorologia1_igm
        ,sorologia1_igg
        ,sorologia1_tit1
        ,resultado_sorologia2_chikungunya
        ,sorologia2_igm
        ,sorologia2_igg
        ,sorologia2_tit1
        ,resultado_prnt
        ,data_ns1
        ,resultado_ns1
        ,data_viral
        ,resultado_viral
        ,data_pcr
        ,resultado_pcr
        ,amostra_pcr
        ,amostra_outra
        ,tecnica
        ,resultado_amostra_outra
        ,data_sorologia_dengue
        ,resultado_sorologia_dengue
        ,sorotipo
        ,histopatologia
        ,imunohistoquimica
        ,manifestacao_hemorragica
        ,classificacao_final
        ,criterio_confirmacao
        ,caso_fhd
        ,caso_autoctone
        ,pais_infeccao
        --,sigla_uf_infeccao
        --,sigla_uf_infeccao_nome
        ,id_municipio_infeccao
        --,id_municipio_infeccao_nome
        ,doenca_trabalho
        ,apresentacao_clinica
        ,evolucao_caso
        --,data_obito
        --,data_encerramento
        ,tipo_sistema
        --,data_digitacao
,  (CASE WHEN LOWER(classificacao_final) LIKE '%dengue%' THEN 1 ELSE 0 END) AS target
,  (CASE WHEN LOWER(evolucao_caso) LIKE '%óbito%' THEN 1 ELSE 0 END) AS obito
,  (CASE WHEN data_encerramento is not null THEN 1  ELSE 0  END) AS flag_caso_encerrado

---Dias
, (CASE 
        WHEN data_encerramento is not null and data_encerramento>=coalesce(data_primeiros_sintomas,data_notificacao)
        THEN  DATE_DIFF(data_encerramento, coalesce(data_primeiros_sintomas,data_notificacao), DAY)
        ELSE null  END) as qt_dias_encerramento
, (CASE 
        WHEN data_obito is not null
        THEN  DATE_DIFF(data_obito, coalesce(data_primeiros_sintomas,data_notificacao), DAY)
        ELSE null  END) as qt_dias_obito
, (CASE 
        WHEN data_primeiros_sintomas is not null
        THEN  DATE_DIFF(data_primeiros_sintomas, data_notificacao, DAY)
        ELSE null  END) as qt_dias_notificacao_primeiros_sintomas
,count(*) as qt

from `basedosdados.br_ms_sinan.microdados_dengue`
where 1=1
and ano=2022
group by 
                tipo_notificacao
                ,id_agravo
                ,data_notificacao
                --,semana_notificacao
                --,sigla_uf_notificacao
                --,sigla_uf_notificacao_nome
                ,id_regional_saude_notificacao
                ,id_municipio_notificacao
                --,id_municipio_notificacao_nome
                ,id_estabelecimento
                ,data_primeiros_sintomas
                ,semana_sintomas
                ,pais_residencia
                --,sigla_uf_residencia
                --,sigla_uf_residencia_nome
                --,id_regional_saude_residencia
                --,id_municipio_residencia
                --,id_municipio_residencia_nome
                ,ano_nascimento_paciente
                ,data_nascimento_paciente
                ,idade_paciente
                ,sexo_paciente
                ,raca_cor_paciente
                ,escolaridade_paciente
                ,ocupacao_paciente 
                ,gestante_paciente
                ,possui_doenca_autoimune
                ,possui_diabetes
                ,possui_doencas_hematologicas
                ,possui_hepatopatias
                ,possui_doenca_renal
                ,possui_hipertensao
                ,possui_doenca_acido_peptica
                ,paciente_vacinado
                --,data_vacina
                ,data_investigacao
                ,apresenta_febre
                ,data_febre
                ,duracao_febre
                ,apresenta_cefaleia
                ,apresenta_exantema
                ,apresenta_dor_costas
                ,apresenta_prostacao
                ,apresenta_mialgia
                ,apresenta_vomito
                ,apresenta_nausea
                ,apresenta_diarreia
                ,apresenta_conjutivite
                ,apresenta_dor_retroorbital
                ,apresenta_artralgia
                ,apresenta_artrite
                ,apresenta_leucopenia
                ,apresenta_epistaxe
                ,apresenta_petequias
                ,apresenta_gengivorragia
                ,apresenta_metrorragia
                ,apresenta_hematuria
                ,apresenta_sangramento
                ,apresenta_complicacao
                ,apresenta_ascite
                ,apresenta_pleurite
                ,apresenta_pericardite
                ,apresenta_dor_abdominal
                ,apresenta_hepatomegalia
                ,apresenta_miocardite
                ,apresenta_hipotensao
                ,apresenta_choque
                ,apresenta_insuficiencia_orgao
                ,apresenta_sintoma_outro
                ,apresenta_qual_sintoma
                ,prova_laco
                ,internacao
                ,data_internacao
                --,sigla_uf_internacao
                --,sigla_uf_internacao_nome
                ,id_municipio_internacao
                --,id_municipio_internacao_nome
                ,alarme_hipotensao
                ,alarme_plaqueta
                ,alarme_vomito
                ,alarme_sangramento
                ,alarme_hematocrito
                ,alarme_dor_abdominal
                ,alarme_letargia
                ,alarme_hepatomegalia
                ,alarme_liquidos
                ,data_alarme
                ,grave_pulso
                ,grave_convulsao
                ,grave_enchimento_capilar
                ,grave_insuficiencia_respiratoria
                ,grave_taquicardia
                ,grave_extremidade_fria
                ,grave_hipotensao
                ,grave_hematemese
                ,grave_melena
                ,grave_metrorragia
                ,grave_sangramento
                ,grave_ast_alt
                ,grave_miocardite
                ,grave_consciencia
                ,grave_orgaos
                ,data_hematocrito
                ,hematocrito_maior
                ,data_plaquetas
                ,plaqueta_maior
                ,data_hematocrito_2
                ,hematocrito_menor
                ,data_plaquetas_2
                ,plaqueta_menor
                ,data_sorologia1_chikungunya
                ,data_resultado_sorologia1_chikungunya
                ,resultado_sorologia1_chikungunya
                ,sorologia1_igm
                ,sorologia1_igg
                ,sorologia1_tit1
                ,resultado_sorologia2_chikungunya
                ,sorologia2_igm
                ,sorologia2_igg
                ,sorologia2_tit1
                ,resultado_prnt
                ,data_ns1
                ,resultado_ns1
                ,data_viral
                ,resultado_viral
                ,data_pcr
                ,resultado_pcr
                ,amostra_pcr
                ,amostra_outra
                ,tecnica
                ,resultado_amostra_outra
                ,data_sorologia_dengue
                ,resultado_sorologia_dengue
                ,sorotipo
                ,histopatologia
                ,imunohistoquimica
                ,manifestacao_hemorragica
                ,classificacao_final
                ,criterio_confirmacao
                ,caso_fhd
                ,caso_autoctone
                ,pais_infeccao
                --,sigla_uf_infeccao
                --,sigla_uf_infeccao_nome
                ,id_municipio_infeccao
                --,id_municipio_infeccao_nome
                ,doenca_trabalho
                ,apresentacao_clinica
                ,evolucao_caso
                ,data_obito
                ,data_encerramento
                ,tipo_sistema
                --,data_digitacao
        ,qt_dias_encerramento
        ,qt_dias_obito
        ,qt_dias_notificacao_primeiros_sintomas


Running query in 'bigquery://ufrgs-ppgc-dengue'

1392704 rows affected.

In [11]:
save_parquet_in_chunks(df,"data/4_gold/sinan/fat_sinan/2022",50,compression='brotli')

Salvando arquivos Parquet:   0%|          | 0/1392704 [00:00<?, ?linhas/s]

✔️ 2022_part_1.parquet salvo com 22.08 MB (1392704 linhas)
✅ Todos os arquivos salvos com sucesso.


## 2023

In [12]:
%%sql df <<
SELECT 
	tipo_notificacao
        ,id_agravo
        ,data_notificacao
        --,semana_notificacao
        --,sigla_uf_notificacao
        --,sigla_uf_notificacao_nome
        ,id_regional_saude_notificacao
        ,id_municipio_notificacao
        --,id_municipio_notificacao_nome
        ,id_estabelecimento as id_cnes
        ,data_primeiros_sintomas
        ,semana_sintomas
        ,pais_residencia
        --,sigla_uf_residencia
        --,sigla_uf_residencia_nome
        --,id_regional_saude_residencia
        --,id_municipio_residencia
        --,id_municipio_residencia_nome
        ,ano_nascimento_paciente
        ,data_nascimento_paciente
        ,idade_paciente
        ,sexo_paciente
        ,raca_cor_paciente
        ,escolaridade_paciente
        ,ocupacao_paciente 
        ,gestante_paciente
        ,possui_doenca_autoimune
        ,possui_diabetes
        ,possui_doencas_hematologicas
        ,possui_hepatopatias
        ,possui_doenca_renal
        ,possui_hipertensao
        ,possui_doenca_acido_peptica
        ,paciente_vacinado
        --,data_vacina
        ,data_investigacao
        ,apresenta_febre
        ,data_febre
        ,duracao_febre
        ,apresenta_cefaleia
        ,apresenta_exantema
        ,apresenta_dor_costas
        ,apresenta_prostacao
        ,apresenta_mialgia
        ,apresenta_vomito
        ,apresenta_nausea
        ,apresenta_diarreia
        ,apresenta_conjutivite
        ,apresenta_dor_retroorbital
        ,apresenta_artralgia
        ,apresenta_artrite
        ,apresenta_leucopenia
        ,apresenta_epistaxe
        ,apresenta_petequias
        ,apresenta_gengivorragia
        ,apresenta_metrorragia
        ,apresenta_hematuria
        ,apresenta_sangramento
        ,apresenta_complicacao
        ,apresenta_ascite
        ,apresenta_pleurite
        ,apresenta_pericardite
        ,apresenta_dor_abdominal
        ,apresenta_hepatomegalia
        ,apresenta_miocardite
        ,apresenta_hipotensao
        ,apresenta_choque
        ,apresenta_insuficiencia_orgao
        ,apresenta_sintoma_outro
        ,apresenta_qual_sintoma
        ,prova_laco
        ,internacao
        ,data_internacao
        --,sigla_uf_internacao
        --,sigla_uf_internacao_nome
        ,id_municipio_internacao
        --,id_municipio_internacao_nome
        ,alarme_hipotensao
        ,alarme_plaqueta
        ,alarme_vomito
        ,alarme_sangramento
        ,alarme_hematocrito
        ,alarme_dor_abdominal
        ,alarme_letargia
        ,alarme_hepatomegalia
        ,alarme_liquidos
        ,data_alarme
        ,grave_pulso
        ,grave_convulsao
        ,grave_enchimento_capilar
        ,grave_insuficiencia_respiratoria
        ,grave_taquicardia
        ,grave_extremidade_fria
        ,grave_hipotensao
        ,grave_hematemese
        ,grave_melena
        ,grave_metrorragia
        ,grave_sangramento
        ,grave_ast_alt
        ,grave_miocardite
        ,grave_consciencia
        ,grave_orgaos
        ,data_hematocrito
        ,hematocrito_maior
        ,data_plaquetas
        ,plaqueta_maior
        ,data_hematocrito_2
        ,hematocrito_menor
        ,data_plaquetas_2
        ,plaqueta_menor
        ,data_sorologia1_chikungunya
        ,data_resultado_sorologia1_chikungunya
        ,resultado_sorologia1_chikungunya
        ,sorologia1_igm
        ,sorologia1_igg
        ,sorologia1_tit1
        ,resultado_sorologia2_chikungunya
        ,sorologia2_igm
        ,sorologia2_igg
        ,sorologia2_tit1
        ,resultado_prnt
        ,data_ns1
        ,resultado_ns1
        ,data_viral
        ,resultado_viral
        ,data_pcr
        ,resultado_pcr
        ,amostra_pcr
        ,amostra_outra
        ,tecnica
        ,resultado_amostra_outra
        ,data_sorologia_dengue
        ,resultado_sorologia_dengue
        ,sorotipo
        ,histopatologia
        ,imunohistoquimica
        ,manifestacao_hemorragica
        ,classificacao_final
        ,criterio_confirmacao
        ,caso_fhd
        ,caso_autoctone
        ,pais_infeccao
        --,sigla_uf_infeccao
        --,sigla_uf_infeccao_nome
        ,id_municipio_infeccao
        --,id_municipio_infeccao_nome
        ,doenca_trabalho
        ,apresentacao_clinica
        ,evolucao_caso
        --,data_obito
        --,data_encerramento
        ,tipo_sistema
        --,data_digitacao
,  (CASE WHEN LOWER(classificacao_final) LIKE '%dengue%' THEN 1 ELSE 0 END) AS target
,  (CASE WHEN LOWER(evolucao_caso) LIKE '%óbito%' THEN 1 ELSE 0 END) AS obito
,  (CASE WHEN data_encerramento is not null THEN 1  ELSE 0  END) AS flag_caso_encerrado

---Dias
, (CASE 
        WHEN data_encerramento is not null and data_encerramento>=coalesce(data_primeiros_sintomas,data_notificacao)
        THEN  DATE_DIFF(data_encerramento, coalesce(data_primeiros_sintomas,data_notificacao), DAY)
        ELSE null  END) as qt_dias_encerramento
, (CASE 
        WHEN data_obito is not null
        THEN  DATE_DIFF(data_obito, coalesce(data_primeiros_sintomas,data_notificacao), DAY)
        ELSE null  END) as qt_dias_obito
, (CASE 
        WHEN data_primeiros_sintomas is not null
        THEN  DATE_DIFF(data_primeiros_sintomas, data_notificacao, DAY)
        ELSE null  END) as qt_dias_notificacao_primeiros_sintomas
,count(*) as qt

from `basedosdados.br_ms_sinan.microdados_dengue`
where 1=1
and ano=2023
group by 
                tipo_notificacao
                ,id_agravo
                ,data_notificacao
                --,semana_notificacao
                --,sigla_uf_notificacao
                --,sigla_uf_notificacao_nome
                ,id_regional_saude_notificacao
                ,id_municipio_notificacao
                --,id_municipio_notificacao_nome
                ,id_estabelecimento
                ,data_primeiros_sintomas
                ,semana_sintomas
                ,pais_residencia
                --,sigla_uf_residencia
                --,sigla_uf_residencia_nome
                --,id_regional_saude_residencia
                --,id_municipio_residencia
                --,id_municipio_residencia_nome
                ,ano_nascimento_paciente
                ,data_nascimento_paciente
                ,idade_paciente
                ,sexo_paciente
                ,raca_cor_paciente
                ,escolaridade_paciente
                ,ocupacao_paciente 
                ,gestante_paciente
                ,possui_doenca_autoimune
                ,possui_diabetes
                ,possui_doencas_hematologicas
                ,possui_hepatopatias
                ,possui_doenca_renal
                ,possui_hipertensao
                ,possui_doenca_acido_peptica
                ,paciente_vacinado
                --,data_vacina
                ,data_investigacao
                ,apresenta_febre
                ,data_febre
                ,duracao_febre
                ,apresenta_cefaleia
                ,apresenta_exantema
                ,apresenta_dor_costas
                ,apresenta_prostacao
                ,apresenta_mialgia
                ,apresenta_vomito
                ,apresenta_nausea
                ,apresenta_diarreia
                ,apresenta_conjutivite
                ,apresenta_dor_retroorbital
                ,apresenta_artralgia
                ,apresenta_artrite
                ,apresenta_leucopenia
                ,apresenta_epistaxe
                ,apresenta_petequias
                ,apresenta_gengivorragia
                ,apresenta_metrorragia
                ,apresenta_hematuria
                ,apresenta_sangramento
                ,apresenta_complicacao
                ,apresenta_ascite
                ,apresenta_pleurite
                ,apresenta_pericardite
                ,apresenta_dor_abdominal
                ,apresenta_hepatomegalia
                ,apresenta_miocardite
                ,apresenta_hipotensao
                ,apresenta_choque
                ,apresenta_insuficiencia_orgao
                ,apresenta_sintoma_outro
                ,apresenta_qual_sintoma
                ,prova_laco
                ,internacao
                ,data_internacao
                --,sigla_uf_internacao
                --,sigla_uf_internacao_nome
                ,id_municipio_internacao
                --,id_municipio_internacao_nome
                ,alarme_hipotensao
                ,alarme_plaqueta
                ,alarme_vomito
                ,alarme_sangramento
                ,alarme_hematocrito
                ,alarme_dor_abdominal
                ,alarme_letargia
                ,alarme_hepatomegalia
                ,alarme_liquidos
                ,data_alarme
                ,grave_pulso
                ,grave_convulsao
                ,grave_enchimento_capilar
                ,grave_insuficiencia_respiratoria
                ,grave_taquicardia
                ,grave_extremidade_fria
                ,grave_hipotensao
                ,grave_hematemese
                ,grave_melena
                ,grave_metrorragia
                ,grave_sangramento
                ,grave_ast_alt
                ,grave_miocardite
                ,grave_consciencia
                ,grave_orgaos
                ,data_hematocrito
                ,hematocrito_maior
                ,data_plaquetas
                ,plaqueta_maior
                ,data_hematocrito_2
                ,hematocrito_menor
                ,data_plaquetas_2
                ,plaqueta_menor
                ,data_sorologia1_chikungunya
                ,data_resultado_sorologia1_chikungunya
                ,resultado_sorologia1_chikungunya
                ,sorologia1_igm
                ,sorologia1_igg
                ,sorologia1_tit1
                ,resultado_sorologia2_chikungunya
                ,sorologia2_igm
                ,sorologia2_igg
                ,sorologia2_tit1
                ,resultado_prnt
                ,data_ns1
                ,resultado_ns1
                ,data_viral
                ,resultado_viral
                ,data_pcr
                ,resultado_pcr
                ,amostra_pcr
                ,amostra_outra
                ,tecnica
                ,resultado_amostra_outra
                ,data_sorologia_dengue
                ,resultado_sorologia_dengue
                ,sorotipo
                ,histopatologia
                ,imunohistoquimica
                ,manifestacao_hemorragica
                ,classificacao_final
                ,criterio_confirmacao
                ,caso_fhd
                ,caso_autoctone
                ,pais_infeccao
                --,sigla_uf_infeccao
                --,sigla_uf_infeccao_nome
                ,id_municipio_infeccao
                --,id_municipio_infeccao_nome
                ,doenca_trabalho
                ,apresentacao_clinica
                ,evolucao_caso
                ,data_obito
                ,data_encerramento
                ,tipo_sistema
                --,data_digitacao
        ,qt_dias_encerramento
        ,qt_dias_obito
        ,qt_dias_notificacao_primeiros_sintomas


Running query in 'bigquery://ufrgs-ppgc-dengue'

1507364 rows affected.

In [13]:
save_parquet_in_chunks(df,"data/4_gold/sinan/fat_sinan/2023",50,compression='brotli')

Salvando arquivos Parquet:   0%|          | 0/1507364 [00:00<?, ?linhas/s]

✔️ 2023_part_1.parquet salvo com 23.83 MB (1507364 linhas)
✅ Todos os arquivos salvos com sucesso.


## 2024

In [5]:
%%sql df <<
SELECT 
	tipo_notificacao
        ,id_agravo
        ,data_notificacao
        --,semana_notificacao
        --,sigla_uf_notificacao
        --,sigla_uf_notificacao_nome
        ,id_regional_saude_notificacao
        ,id_municipio_notificacao
        --,id_municipio_notificacao_nome
        ,id_estabelecimento as id_cnes
        ,data_primeiros_sintomas
        ,semana_sintomas
        ,pais_residencia
        --,sigla_uf_residencia
        --,sigla_uf_residencia_nome
        --,id_regional_saude_residencia
        --,id_municipio_residencia
        --,id_municipio_residencia_nome
        ,ano_nascimento_paciente
        ,data_nascimento_paciente
        ,idade_paciente
        ,sexo_paciente
        ,raca_cor_paciente
        ,escolaridade_paciente
        ,ocupacao_paciente 
        ,gestante_paciente
        ,possui_doenca_autoimune
        ,possui_diabetes
        ,possui_doencas_hematologicas
        ,possui_hepatopatias
        ,possui_doenca_renal
        ,possui_hipertensao
        ,possui_doenca_acido_peptica
        ,paciente_vacinado
        --,data_vacina
        ,data_investigacao
        ,apresenta_febre
        ,data_febre
        ,duracao_febre
        ,apresenta_cefaleia
        ,apresenta_exantema
        ,apresenta_dor_costas
        ,apresenta_prostacao
        ,apresenta_mialgia
        ,apresenta_vomito
        ,apresenta_nausea
        ,apresenta_diarreia
        ,apresenta_conjutivite
        ,apresenta_dor_retroorbital
        ,apresenta_artralgia
        ,apresenta_artrite
        ,apresenta_leucopenia
        ,apresenta_epistaxe
        ,apresenta_petequias
        ,apresenta_gengivorragia
        ,apresenta_metrorragia
        ,apresenta_hematuria
        ,apresenta_sangramento
        ,apresenta_complicacao
        ,apresenta_ascite
        ,apresenta_pleurite
        ,apresenta_pericardite
        ,apresenta_dor_abdominal
        ,apresenta_hepatomegalia
        ,apresenta_miocardite
        ,apresenta_hipotensao
        ,apresenta_choque
        ,apresenta_insuficiencia_orgao
        ,apresenta_sintoma_outro
        ,apresenta_qual_sintoma
        ,prova_laco
        ,internacao
        ,data_internacao
        --,sigla_uf_internacao
        --,sigla_uf_internacao_nome
        ,id_municipio_internacao
        --,id_municipio_internacao_nome
        ,alarme_hipotensao
        ,alarme_plaqueta
        ,alarme_vomito
        ,alarme_sangramento
        ,alarme_hematocrito
        ,alarme_dor_abdominal
        ,alarme_letargia
        ,alarme_hepatomegalia
        ,alarme_liquidos
        ,data_alarme
        ,grave_pulso
        ,grave_convulsao
        ,grave_enchimento_capilar
        ,grave_insuficiencia_respiratoria
        ,grave_taquicardia
        ,grave_extremidade_fria
        ,grave_hipotensao
        ,grave_hematemese
        ,grave_melena
        ,grave_metrorragia
        ,grave_sangramento
        ,grave_ast_alt
        ,grave_miocardite
        ,grave_consciencia
        ,grave_orgaos
        ,data_hematocrito
        ,hematocrito_maior
        ,data_plaquetas
        ,plaqueta_maior
        ,data_hematocrito_2
        ,hematocrito_menor
        ,data_plaquetas_2
        ,plaqueta_menor
        ,data_sorologia1_chikungunya
        ,data_resultado_sorologia1_chikungunya
        ,resultado_sorologia1_chikungunya
        ,sorologia1_igm
        ,sorologia1_igg
        ,sorologia1_tit1
        ,resultado_sorologia2_chikungunya
        ,sorologia2_igm
        ,sorologia2_igg
        ,sorologia2_tit1
        ,resultado_prnt
        ,data_ns1
        ,resultado_ns1
        ,data_viral
        ,resultado_viral
        ,data_pcr
        ,resultado_pcr
        ,amostra_pcr
        ,amostra_outra
        ,tecnica
        ,resultado_amostra_outra
        ,data_sorologia_dengue
        ,resultado_sorologia_dengue
        ,sorotipo
        ,histopatologia
        ,imunohistoquimica
        ,manifestacao_hemorragica
        ,classificacao_final
        ,criterio_confirmacao
        ,caso_fhd
        ,caso_autoctone
        ,pais_infeccao
        --,sigla_uf_infeccao
        --,sigla_uf_infeccao_nome
        ,id_municipio_infeccao
        --,id_municipio_infeccao_nome
        ,doenca_trabalho
        ,apresentacao_clinica
        ,evolucao_caso
        --,data_obito
        --,data_encerramento
        ,tipo_sistema
        --,data_digitacao
,  (CASE WHEN LOWER(classificacao_final) LIKE '%dengue%' THEN 1 ELSE 0 END) AS target
,  (CASE WHEN LOWER(evolucao_caso) LIKE '%óbito%' THEN 1 ELSE 0 END) AS obito
,  (CASE WHEN data_encerramento is not null THEN 1  ELSE 0  END) AS flag_caso_encerrado

---Dias
, (CASE 
        WHEN data_encerramento is not null and data_encerramento>=coalesce(data_primeiros_sintomas,data_notificacao)
        THEN  DATE_DIFF(data_encerramento, coalesce(data_primeiros_sintomas,data_notificacao), DAY)
        ELSE null  END) as qt_dias_encerramento
, (CASE 
        WHEN data_obito is not null
        THEN  DATE_DIFF(data_obito, coalesce(data_primeiros_sintomas,data_notificacao), DAY)
        ELSE null  END) as qt_dias_obito
, (CASE 
        WHEN data_primeiros_sintomas is not null
        THEN  DATE_DIFF(data_primeiros_sintomas, data_notificacao, DAY)
        ELSE null  END) as qt_dias_notificacao_primeiros_sintomas
,count(*) as qt

from `basedosdados.br_ms_sinan.microdados_dengue`
where 1=1
and ano=2024 and data_notificacao<'2024-07-01'
group by 
                tipo_notificacao
                ,id_agravo
                ,data_notificacao
                --,semana_notificacao
                --,sigla_uf_notificacao
                --,sigla_uf_notificacao_nome
                ,id_regional_saude_notificacao
                ,id_municipio_notificacao
                --,id_municipio_notificacao_nome
                ,id_estabelecimento
                ,data_primeiros_sintomas
                ,semana_sintomas
                ,pais_residencia
                --,sigla_uf_residencia
                --,sigla_uf_residencia_nome
                --,id_regional_saude_residencia
                --,id_municipio_residencia
                --,id_municipio_residencia_nome
                ,ano_nascimento_paciente
                ,data_nascimento_paciente
                ,idade_paciente
                ,sexo_paciente
                ,raca_cor_paciente
                ,escolaridade_paciente
                ,ocupacao_paciente 
                ,gestante_paciente
                ,possui_doenca_autoimune
                ,possui_diabetes
                ,possui_doencas_hematologicas
                ,possui_hepatopatias
                ,possui_doenca_renal
                ,possui_hipertensao
                ,possui_doenca_acido_peptica
                ,paciente_vacinado
                --,data_vacina
                ,data_investigacao
                ,apresenta_febre
                ,data_febre
                ,duracao_febre
                ,apresenta_cefaleia
                ,apresenta_exantema
                ,apresenta_dor_costas
                ,apresenta_prostacao
                ,apresenta_mialgia
                ,apresenta_vomito
                ,apresenta_nausea
                ,apresenta_diarreia
                ,apresenta_conjutivite
                ,apresenta_dor_retroorbital
                ,apresenta_artralgia
                ,apresenta_artrite
                ,apresenta_leucopenia
                ,apresenta_epistaxe
                ,apresenta_petequias
                ,apresenta_gengivorragia
                ,apresenta_metrorragia
                ,apresenta_hematuria
                ,apresenta_sangramento
                ,apresenta_complicacao
                ,apresenta_ascite
                ,apresenta_pleurite
                ,apresenta_pericardite
                ,apresenta_dor_abdominal
                ,apresenta_hepatomegalia
                ,apresenta_miocardite
                ,apresenta_hipotensao
                ,apresenta_choque
                ,apresenta_insuficiencia_orgao
                ,apresenta_sintoma_outro
                ,apresenta_qual_sintoma
                ,prova_laco
                ,internacao
                ,data_internacao
                --,sigla_uf_internacao
                --,sigla_uf_internacao_nome
                ,id_municipio_internacao
                --,id_municipio_internacao_nome
                ,alarme_hipotensao
                ,alarme_plaqueta
                ,alarme_vomito
                ,alarme_sangramento
                ,alarme_hematocrito
                ,alarme_dor_abdominal
                ,alarme_letargia
                ,alarme_hepatomegalia
                ,alarme_liquidos
                ,data_alarme
                ,grave_pulso
                ,grave_convulsao
                ,grave_enchimento_capilar
                ,grave_insuficiencia_respiratoria
                ,grave_taquicardia
                ,grave_extremidade_fria
                ,grave_hipotensao
                ,grave_hematemese
                ,grave_melena
                ,grave_metrorragia
                ,grave_sangramento
                ,grave_ast_alt
                ,grave_miocardite
                ,grave_consciencia
                ,grave_orgaos
                ,data_hematocrito
                ,hematocrito_maior
                ,data_plaquetas
                ,plaqueta_maior
                ,data_hematocrito_2
                ,hematocrito_menor
                ,data_plaquetas_2
                ,plaqueta_menor
                ,data_sorologia1_chikungunya
                ,data_resultado_sorologia1_chikungunya
                ,resultado_sorologia1_chikungunya
                ,sorologia1_igm
                ,sorologia1_igg
                ,sorologia1_tit1
                ,resultado_sorologia2_chikungunya
                ,sorologia2_igm
                ,sorologia2_igg
                ,sorologia2_tit1
                ,resultado_prnt
                ,data_ns1
                ,resultado_ns1
                ,data_viral
                ,resultado_viral
                ,data_pcr
                ,resultado_pcr
                ,amostra_pcr
                ,amostra_outra
                ,tecnica
                ,resultado_amostra_outra
                ,data_sorologia_dengue
                ,resultado_sorologia_dengue
                ,sorotipo
                ,histopatologia
                ,imunohistoquimica
                ,manifestacao_hemorragica
                ,classificacao_final
                ,criterio_confirmacao
                ,caso_fhd
                ,caso_autoctone
                ,pais_infeccao
                --,sigla_uf_infeccao
                --,sigla_uf_infeccao_nome
                ,id_municipio_infeccao
                --,id_municipio_infeccao_nome
                ,doenca_trabalho
                ,apresentacao_clinica
                ,evolucao_caso
                ,data_obito
                ,data_encerramento
                ,tipo_sistema
                --,data_digitacao
        ,qt_dias_encerramento
        ,qt_dias_obito
        ,qt_dias_notificacao_primeiros_sintomas


Running query in 'bigquery://ufrgs-ppgc-dengue'

6078756 rows affected.

In [6]:
save_parquet_in_chunks(df,"data/4_gold/sinan/fat_sinan/2024_1",50,compression='brotli')

Salvando arquivos Parquet:   0%|          | 0/6078756 [00:00<?, ?linhas/s]

✔️ 2024_1_part_1.parquet salvo com 50.00 MB (2830107 linhas)
✔️ 2024_1_part_2.parquet salvo com 50.00 MB (2825650 linhas)
✔️ 2024_1_part_3.parquet salvo com 7.58 MB (422999 linhas)
✅ Todos os arquivos salvos com sucesso.


In [7]:
del(df)

In [7]:
%%sql df <<
SELECT 
	tipo_notificacao
        ,id_agravo
        ,data_notificacao
        --,semana_notificacao
        --,sigla_uf_notificacao
        --,sigla_uf_notificacao_nome
        ,id_regional_saude_notificacao
        ,id_municipio_notificacao
        --,id_municipio_notificacao_nome
        ,id_estabelecimento as id_cnes
        ,data_primeiros_sintomas
        ,semana_sintomas
        ,pais_residencia
        --,sigla_uf_residencia
        --,sigla_uf_residencia_nome
        --,id_regional_saude_residencia
        --,id_municipio_residencia
        --,id_municipio_residencia_nome
        ,ano_nascimento_paciente
        ,data_nascimento_paciente
        ,idade_paciente
        ,sexo_paciente
        ,raca_cor_paciente
        ,escolaridade_paciente
        ,ocupacao_paciente 
        ,gestante_paciente
        ,possui_doenca_autoimune
        ,possui_diabetes
        ,possui_doencas_hematologicas
        ,possui_hepatopatias
        ,possui_doenca_renal
        ,possui_hipertensao
        ,possui_doenca_acido_peptica
        ,paciente_vacinado
        --,data_vacina
        ,data_investigacao
        ,apresenta_febre
        ,data_febre
        ,duracao_febre
        ,apresenta_cefaleia
        ,apresenta_exantema
        ,apresenta_dor_costas
        ,apresenta_prostacao
        ,apresenta_mialgia
        ,apresenta_vomito
        ,apresenta_nausea
        ,apresenta_diarreia
        ,apresenta_conjutivite
        ,apresenta_dor_retroorbital
        ,apresenta_artralgia
        ,apresenta_artrite
        ,apresenta_leucopenia
        ,apresenta_epistaxe
        ,apresenta_petequias
        ,apresenta_gengivorragia
        ,apresenta_metrorragia
        ,apresenta_hematuria
        ,apresenta_sangramento
        ,apresenta_complicacao
        ,apresenta_ascite
        ,apresenta_pleurite
        ,apresenta_pericardite
        ,apresenta_dor_abdominal
        ,apresenta_hepatomegalia
        ,apresenta_miocardite
        ,apresenta_hipotensao
        ,apresenta_choque
        ,apresenta_insuficiencia_orgao
        ,apresenta_sintoma_outro
        ,apresenta_qual_sintoma
        ,prova_laco
        ,internacao
        ,data_internacao
        --,sigla_uf_internacao
        --,sigla_uf_internacao_nome
        ,id_municipio_internacao
        --,id_municipio_internacao_nome
        ,alarme_hipotensao
        ,alarme_plaqueta
        ,alarme_vomito
        ,alarme_sangramento
        ,alarme_hematocrito
        ,alarme_dor_abdominal
        ,alarme_letargia
        ,alarme_hepatomegalia
        ,alarme_liquidos
        ,data_alarme
        ,grave_pulso
        ,grave_convulsao
        ,grave_enchimento_capilar
        ,grave_insuficiencia_respiratoria
        ,grave_taquicardia
        ,grave_extremidade_fria
        ,grave_hipotensao
        ,grave_hematemese
        ,grave_melena
        ,grave_metrorragia
        ,grave_sangramento
        ,grave_ast_alt
        ,grave_miocardite
        ,grave_consciencia
        ,grave_orgaos
        ,data_hematocrito
        ,hematocrito_maior
        ,data_plaquetas
        ,plaqueta_maior
        ,data_hematocrito_2
        ,hematocrito_menor
        ,data_plaquetas_2
        ,plaqueta_menor
        ,data_sorologia1_chikungunya
        ,data_resultado_sorologia1_chikungunya
        ,resultado_sorologia1_chikungunya
        ,sorologia1_igm
        ,sorologia1_igg
        ,sorologia1_tit1
        ,resultado_sorologia2_chikungunya
        ,sorologia2_igm
        ,sorologia2_igg
        ,sorologia2_tit1
        ,resultado_prnt
        ,data_ns1
        ,resultado_ns1
        ,data_viral
        ,resultado_viral
        ,data_pcr
        ,resultado_pcr
        ,amostra_pcr
        ,amostra_outra
        ,tecnica
        ,resultado_amostra_outra
        ,data_sorologia_dengue
        ,resultado_sorologia_dengue
        ,sorotipo
        ,histopatologia
        ,imunohistoquimica
        ,manifestacao_hemorragica
        ,classificacao_final
        ,criterio_confirmacao
        ,caso_fhd
        ,caso_autoctone
        ,pais_infeccao
        --,sigla_uf_infeccao
        --,sigla_uf_infeccao_nome
        ,id_municipio_infeccao
        --,id_municipio_infeccao_nome
        ,doenca_trabalho
        ,apresentacao_clinica
        ,evolucao_caso
        --,data_obito
        --,data_encerramento
        ,tipo_sistema
        --,data_digitacao
,  (CASE WHEN LOWER(classificacao_final) LIKE '%dengue%' THEN 1 ELSE 0 END) AS target
,  (CASE WHEN LOWER(evolucao_caso) LIKE '%óbito%' THEN 1 ELSE 0 END) AS obito
,  (CASE WHEN data_encerramento is not null THEN 1  ELSE 0  END) AS flag_caso_encerrado

---Dias
, (CASE 
        WHEN data_encerramento is not null and data_encerramento>=coalesce(data_primeiros_sintomas,data_notificacao)
        THEN  DATE_DIFF(data_encerramento, coalesce(data_primeiros_sintomas,data_notificacao), DAY)
        ELSE null  END) as qt_dias_encerramento
, (CASE 
        WHEN data_obito is not null
        THEN  DATE_DIFF(data_obito, coalesce(data_primeiros_sintomas,data_notificacao), DAY)
        ELSE null  END) as qt_dias_obito
, (CASE 
        WHEN data_primeiros_sintomas is not null
        THEN  DATE_DIFF(data_primeiros_sintomas, data_notificacao, DAY)
        ELSE null  END) as qt_dias_notificacao_primeiros_sintomas
,count(*) as qt

from `basedosdados.br_ms_sinan.microdados_dengue`
where 1=1
and ano=2024 and data_notificacao>='2024-07-01'
group by 
                tipo_notificacao
                ,id_agravo
                ,data_notificacao
                --,semana_notificacao
                --,sigla_uf_notificacao
                --,sigla_uf_notificacao_nome
                ,id_regional_saude_notificacao
                ,id_municipio_notificacao
                --,id_municipio_notificacao_nome
                ,id_estabelecimento
                ,data_primeiros_sintomas
                ,semana_sintomas
                ,pais_residencia
                --,sigla_uf_residencia
                --,sigla_uf_residencia_nome
                --,id_regional_saude_residencia
                --,id_municipio_residencia
                --,id_municipio_residencia_nome
                ,ano_nascimento_paciente
                ,data_nascimento_paciente
                ,idade_paciente
                ,sexo_paciente
                ,raca_cor_paciente
                ,escolaridade_paciente
                ,ocupacao_paciente 
                ,gestante_paciente
                ,possui_doenca_autoimune
                ,possui_diabetes
                ,possui_doencas_hematologicas
                ,possui_hepatopatias
                ,possui_doenca_renal
                ,possui_hipertensao
                ,possui_doenca_acido_peptica
                ,paciente_vacinado
                --,data_vacina
                ,data_investigacao
                ,apresenta_febre
                ,data_febre
                ,duracao_febre
                ,apresenta_cefaleia
                ,apresenta_exantema
                ,apresenta_dor_costas
                ,apresenta_prostacao
                ,apresenta_mialgia
                ,apresenta_vomito
                ,apresenta_nausea
                ,apresenta_diarreia
                ,apresenta_conjutivite
                ,apresenta_dor_retroorbital
                ,apresenta_artralgia
                ,apresenta_artrite
                ,apresenta_leucopenia
                ,apresenta_epistaxe
                ,apresenta_petequias
                ,apresenta_gengivorragia
                ,apresenta_metrorragia
                ,apresenta_hematuria
                ,apresenta_sangramento
                ,apresenta_complicacao
                ,apresenta_ascite
                ,apresenta_pleurite
                ,apresenta_pericardite
                ,apresenta_dor_abdominal
                ,apresenta_hepatomegalia
                ,apresenta_miocardite
                ,apresenta_hipotensao
                ,apresenta_choque
                ,apresenta_insuficiencia_orgao
                ,apresenta_sintoma_outro
                ,apresenta_qual_sintoma
                ,prova_laco
                ,internacao
                ,data_internacao
                --,sigla_uf_internacao
                --,sigla_uf_internacao_nome
                ,id_municipio_internacao
                --,id_municipio_internacao_nome
                ,alarme_hipotensao
                ,alarme_plaqueta
                ,alarme_vomito
                ,alarme_sangramento
                ,alarme_hematocrito
                ,alarme_dor_abdominal
                ,alarme_letargia
                ,alarme_hepatomegalia
                ,alarme_liquidos
                ,data_alarme
                ,grave_pulso
                ,grave_convulsao
                ,grave_enchimento_capilar
                ,grave_insuficiencia_respiratoria
                ,grave_taquicardia
                ,grave_extremidade_fria
                ,grave_hipotensao
                ,grave_hematemese
                ,grave_melena
                ,grave_metrorragia
                ,grave_sangramento
                ,grave_ast_alt
                ,grave_miocardite
                ,grave_consciencia
                ,grave_orgaos
                ,data_hematocrito
                ,hematocrito_maior
                ,data_plaquetas
                ,plaqueta_maior
                ,data_hematocrito_2
                ,hematocrito_menor
                ,data_plaquetas_2
                ,plaqueta_menor
                ,data_sorologia1_chikungunya
                ,data_resultado_sorologia1_chikungunya
                ,resultado_sorologia1_chikungunya
                ,sorologia1_igm
                ,sorologia1_igg
                ,sorologia1_tit1
                ,resultado_sorologia2_chikungunya
                ,sorologia2_igm
                ,sorologia2_igg
                ,sorologia2_tit1
                ,resultado_prnt
                ,data_ns1
                ,resultado_ns1
                ,data_viral
                ,resultado_viral
                ,data_pcr
                ,resultado_pcr
                ,amostra_pcr
                ,amostra_outra
                ,tecnica
                ,resultado_amostra_outra
                ,data_sorologia_dengue
                ,resultado_sorologia_dengue
                ,sorotipo
                ,histopatologia
                ,imunohistoquimica
                ,manifestacao_hemorragica
                ,classificacao_final
                ,criterio_confirmacao
                ,caso_fhd
                ,caso_autoctone
                ,pais_infeccao
                --,sigla_uf_infeccao
                --,sigla_uf_infeccao_nome
                ,id_municipio_infeccao
                --,id_municipio_infeccao_nome
                ,doenca_trabalho
                ,apresentacao_clinica
                ,evolucao_caso
                ,data_obito
                ,data_encerramento
                ,tipo_sistema
                --,data_digitacao
        ,qt_dias_encerramento
        ,qt_dias_obito
        ,qt_dias_notificacao_primeiros_sintomas


Running query in 'bigquery://ufrgs-ppgc-dengue'

349199 rows affected.

In [8]:
save_parquet_in_chunks(df,"data/4_gold/sinan/fat_sinan/2024_2",50,compression='brotli')

Salvando arquivos Parquet:   0%|          | 0/349199 [00:00<?, ?linhas/s]

✔️ 2024_2_part_1.parquet salvo com 5.57 MB (349199 linhas)
✅ Todos os arquivos salvos com sucesso.


## 2025

In [5]:
%%sql df <<
SELECT 
	tipo_notificacao
        ,id_agravo
        ,data_notificacao
        --,semana_notificacao
        --,sigla_uf_notificacao
        --,sigla_uf_notificacao_nome
        ,id_regional_saude_notificacao
        ,id_municipio_notificacao
        --,id_municipio_notificacao_nome
        ,id_estabelecimento as id_cnes
        ,data_primeiros_sintomas
        ,semana_sintomas
        ,pais_residencia
        --,sigla_uf_residencia
        --,sigla_uf_residencia_nome
        --,id_regional_saude_residencia
        --,id_municipio_residencia
        --,id_municipio_residencia_nome
        ,ano_nascimento_paciente
        ,data_nascimento_paciente
        ,idade_paciente
        ,sexo_paciente
        ,raca_cor_paciente
        ,escolaridade_paciente
        ,ocupacao_paciente 
        ,gestante_paciente
        ,possui_doenca_autoimune
        ,possui_diabetes
        ,possui_doencas_hematologicas
        ,possui_hepatopatias
        ,possui_doenca_renal
        ,possui_hipertensao
        ,possui_doenca_acido_peptica
        ,paciente_vacinado
        --,data_vacina
        ,data_investigacao
        ,apresenta_febre
        ,data_febre
        ,duracao_febre
        ,apresenta_cefaleia
        ,apresenta_exantema
        ,apresenta_dor_costas
        ,apresenta_prostacao
        ,apresenta_mialgia
        ,apresenta_vomito
        ,apresenta_nausea
        ,apresenta_diarreia
        ,apresenta_conjutivite
        ,apresenta_dor_retroorbital
        ,apresenta_artralgia
        ,apresenta_artrite
        ,apresenta_leucopenia
        ,apresenta_epistaxe
        ,apresenta_petequias
        ,apresenta_gengivorragia
        ,apresenta_metrorragia
        ,apresenta_hematuria
        ,apresenta_sangramento
        ,apresenta_complicacao
        ,apresenta_ascite
        ,apresenta_pleurite
        ,apresenta_pericardite
        ,apresenta_dor_abdominal
        ,apresenta_hepatomegalia
        ,apresenta_miocardite
        ,apresenta_hipotensao
        ,apresenta_choque
        ,apresenta_insuficiencia_orgao
        ,apresenta_sintoma_outro
        ,apresenta_qual_sintoma
        ,prova_laco
        ,internacao
        ,data_internacao
        --,sigla_uf_internacao
        --,sigla_uf_internacao_nome
        ,id_municipio_internacao
        --,id_municipio_internacao_nome
        ,alarme_hipotensao
        ,alarme_plaqueta
        ,alarme_vomito
        ,alarme_sangramento
        ,alarme_hematocrito
        ,alarme_dor_abdominal
        ,alarme_letargia
        ,alarme_hepatomegalia
        ,alarme_liquidos
        ,data_alarme
        ,grave_pulso
        ,grave_convulsao
        ,grave_enchimento_capilar
        ,grave_insuficiencia_respiratoria
        ,grave_taquicardia
        ,grave_extremidade_fria
        ,grave_hipotensao
        ,grave_hematemese
        ,grave_melena
        ,grave_metrorragia
        ,grave_sangramento
        ,grave_ast_alt
        ,grave_miocardite
        ,grave_consciencia
        ,grave_orgaos
        ,data_hematocrito
        ,hematocrito_maior
        ,data_plaquetas
        ,plaqueta_maior
        ,data_hematocrito_2
        ,hematocrito_menor
        ,data_plaquetas_2
        ,plaqueta_menor
        ,data_sorologia1_chikungunya
        ,data_resultado_sorologia1_chikungunya
        ,resultado_sorologia1_chikungunya
        ,sorologia1_igm
        ,sorologia1_igg
        ,sorologia1_tit1
        ,resultado_sorologia2_chikungunya
        ,sorologia2_igm
        ,sorologia2_igg
        ,sorologia2_tit1
        ,resultado_prnt
        ,data_ns1
        ,resultado_ns1
        ,data_viral
        ,resultado_viral
        ,data_pcr
        ,resultado_pcr
        ,amostra_pcr
        ,amostra_outra
        ,tecnica
        ,resultado_amostra_outra
        ,data_sorologia_dengue
        ,resultado_sorologia_dengue
        ,sorotipo
        ,histopatologia
        ,imunohistoquimica
        ,manifestacao_hemorragica
        ,classificacao_final
        ,criterio_confirmacao
        ,caso_fhd
        ,caso_autoctone
        ,pais_infeccao
        --,sigla_uf_infeccao
        --,sigla_uf_infeccao_nome
        ,id_municipio_infeccao
        --,id_municipio_infeccao_nome
        ,doenca_trabalho
        ,apresentacao_clinica
        ,evolucao_caso
        --,data_obito
        --,data_encerramento
        ,tipo_sistema
        --,data_digitacao
,  (CASE WHEN LOWER(classificacao_final) LIKE '%dengue%' THEN 1 ELSE 0 END) AS target
,  (CASE WHEN LOWER(evolucao_caso) LIKE '%óbito%' THEN 1 ELSE 0 END) AS obito
,  (CASE WHEN data_encerramento is not null THEN 1  ELSE 0  END) AS flag_caso_encerrado

---Dias
, (CASE 
        WHEN data_encerramento is not null and data_encerramento>=coalesce(data_primeiros_sintomas,data_notificacao)
        THEN  DATE_DIFF(data_encerramento, coalesce(data_primeiros_sintomas,data_notificacao), DAY)
        ELSE null  END) as qt_dias_encerramento
, (CASE 
        WHEN data_obito is not null
        THEN  DATE_DIFF(data_obito, coalesce(data_primeiros_sintomas,data_notificacao), DAY)
        ELSE null  END) as qt_dias_obito
, (CASE 
        WHEN data_primeiros_sintomas is not null
        THEN  DATE_DIFF(data_primeiros_sintomas, data_notificacao, DAY)
        ELSE null  END) as qt_dias_notificacao_primeiros_sintomas
,count(*) as qt

from `basedosdados.br_ms_sinan.microdados_dengue`
where 1=1
and ano=2025
group by 
                tipo_notificacao
                ,id_agravo
                ,data_notificacao
                --,semana_notificacao
                --,sigla_uf_notificacao
                --,sigla_uf_notificacao_nome
                ,id_regional_saude_notificacao
                ,id_municipio_notificacao
                --,id_municipio_notificacao_nome
                ,id_estabelecimento
                ,data_primeiros_sintomas
                ,semana_sintomas
                ,pais_residencia
                --,sigla_uf_residencia
                --,sigla_uf_residencia_nome
                --,id_regional_saude_residencia
                --,id_municipio_residencia
                --,id_municipio_residencia_nome
                ,ano_nascimento_paciente
                ,data_nascimento_paciente
                ,idade_paciente
                ,sexo_paciente
                ,raca_cor_paciente
                ,escolaridade_paciente
                ,ocupacao_paciente 
                ,gestante_paciente
                ,possui_doenca_autoimune
                ,possui_diabetes
                ,possui_doencas_hematologicas
                ,possui_hepatopatias
                ,possui_doenca_renal
                ,possui_hipertensao
                ,possui_doenca_acido_peptica
                ,paciente_vacinado
                --,data_vacina
                ,data_investigacao
                ,apresenta_febre
                ,data_febre
                ,duracao_febre
                ,apresenta_cefaleia
                ,apresenta_exantema
                ,apresenta_dor_costas
                ,apresenta_prostacao
                ,apresenta_mialgia
                ,apresenta_vomito
                ,apresenta_nausea
                ,apresenta_diarreia
                ,apresenta_conjutivite
                ,apresenta_dor_retroorbital
                ,apresenta_artralgia
                ,apresenta_artrite
                ,apresenta_leucopenia
                ,apresenta_epistaxe
                ,apresenta_petequias
                ,apresenta_gengivorragia
                ,apresenta_metrorragia
                ,apresenta_hematuria
                ,apresenta_sangramento
                ,apresenta_complicacao
                ,apresenta_ascite
                ,apresenta_pleurite
                ,apresenta_pericardite
                ,apresenta_dor_abdominal
                ,apresenta_hepatomegalia
                ,apresenta_miocardite
                ,apresenta_hipotensao
                ,apresenta_choque
                ,apresenta_insuficiencia_orgao
                ,apresenta_sintoma_outro
                ,apresenta_qual_sintoma
                ,prova_laco
                ,internacao
                ,data_internacao
                --,sigla_uf_internacao
                --,sigla_uf_internacao_nome
                ,id_municipio_internacao
                --,id_municipio_internacao_nome
                ,alarme_hipotensao
                ,alarme_plaqueta
                ,alarme_vomito
                ,alarme_sangramento
                ,alarme_hematocrito
                ,alarme_dor_abdominal
                ,alarme_letargia
                ,alarme_hepatomegalia
                ,alarme_liquidos
                ,data_alarme
                ,grave_pulso
                ,grave_convulsao
                ,grave_enchimento_capilar
                ,grave_insuficiencia_respiratoria
                ,grave_taquicardia
                ,grave_extremidade_fria
                ,grave_hipotensao
                ,grave_hematemese
                ,grave_melena
                ,grave_metrorragia
                ,grave_sangramento
                ,grave_ast_alt
                ,grave_miocardite
                ,grave_consciencia
                ,grave_orgaos
                ,data_hematocrito
                ,hematocrito_maior
                ,data_plaquetas
                ,plaqueta_maior
                ,data_hematocrito_2
                ,hematocrito_menor
                ,data_plaquetas_2
                ,plaqueta_menor
                ,data_sorologia1_chikungunya
                ,data_resultado_sorologia1_chikungunya
                ,resultado_sorologia1_chikungunya
                ,sorologia1_igm
                ,sorologia1_igg
                ,sorologia1_tit1
                ,resultado_sorologia2_chikungunya
                ,sorologia2_igm
                ,sorologia2_igg
                ,sorologia2_tit1
                ,resultado_prnt
                ,data_ns1
                ,resultado_ns1
                ,data_viral
                ,resultado_viral
                ,data_pcr
                ,resultado_pcr
                ,amostra_pcr
                ,amostra_outra
                ,tecnica
                ,resultado_amostra_outra
                ,data_sorologia_dengue
                ,resultado_sorologia_dengue
                ,sorotipo
                ,histopatologia
                ,imunohistoquimica
                ,manifestacao_hemorragica
                ,classificacao_final
                ,criterio_confirmacao
                ,caso_fhd
                ,caso_autoctone
                ,pais_infeccao
                --,sigla_uf_infeccao
                --,sigla_uf_infeccao_nome
                ,id_municipio_infeccao
                --,id_municipio_infeccao_nome
                ,doenca_trabalho
                ,apresentacao_clinica
                ,evolucao_caso
                ,data_obito
                ,data_encerramento
                ,tipo_sistema
                --,data_digitacao
        ,qt_dias_encerramento
        ,qt_dias_obito
        ,qt_dias_notificacao_primeiros_sintomas


Running query in 'bigquery://ufrgs-ppgc-dengue'

1616395 rows affected.

In [6]:
save_parquet_in_chunks(df,"data/4_gold/sinan/fat_sinan/2025",50,compression='brotli')

Salvando arquivos Parquet:   0%|          | 0/1616395 [00:00<?, ?linhas/s]

✔️ 2025_part_1.parquet salvo com 29.53 MB (1616395 linhas)
✅ Todos os arquivos salvos com sucesso.
